In [1]:
import tensorflow as tf

In [2]:
import os
import zipfile
import keras_preprocessing
from keras_preprocessing import image
from tensorflow.keras import layers
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Input
from tensorflow.keras.models import Model 
from tensorflow.keras.metrics import Precision
from tensorflow.keras.optimizers import RMSprop

In [3]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('val_accuracy')>=0.9700) and (logs.get('val_precision')>=0.9775) and (logs.get('val_mae')<=0.0385)):
      print("\nTa-da! ₍ᐢ•ﻌ•ᐢ₎*･ﾟ｡ Reached 97% accuracy, 97.75% precision, and less than .0385 MAE, so cancelling training!")
      self.model.stop_training = True

In [4]:
callbacks = myCallback()
precision = Precision()

In [5]:
#This block gets the pre-trained model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2021-01-31 20:04:29--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.164.176, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   197MB/s    in 0.4s    

2021-01-31 20:04:29 (197 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [6]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

--2021-01-31 20:04:31--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.176, 172.217.2.112, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   170MB/s    in 0.4s    

2021-01-31 20:04:32 (170 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
# Flatten the output layer to 1 dimension
x = Flatten()(last_output)
x = Dense(644, activation='swish')(x)
x = Dropout(0.33)(x)
x = Dense(644, activation='swish')(x)
x = Dropout(0.33)(x) 
x = Dense(124, activation='swish')(x) 
x = Dropout(0.33)(x)    
x = Dense(62, activation='swish')(x) 
x = Dropout(0.33)(x)               
x = Dense(1, activation='sigmoid')(x) 

model1 = Model(pre_trained_model.input, x) 

In [8]:
#next, create a custom optimizer
optimizer1 = RMSprop(lr=0.0001)


In [9]:
model1.compile(loss='binary_crossentropy',
              optimizer=optimizer1,
              metrics=['mae','accuracy', precision])

history1 = model1.fit(train_generator, 
                      validation_data = validation_generator,
                      epochs=40, 
                      steps_per_epoch = 100,
                      validation_steps = 50,
                      verbose = 2,
                       callbacks=[callbacks])

Epoch 1/40
100/100 - 28s - loss: 0.3427 - mae: 0.2260 - accuracy: 0.8390 - precision: 0.8466 - val_loss: 0.1116 - val_mae: 0.0629 - val_accuracy: 0.9530 - val_precision: 0.9415
Epoch 2/40
100/100 - 17s - loss: 0.2263 - mae: 0.1246 - accuracy: 0.9050 - precision: 0.9108 - val_loss: 0.0993 - val_mae: 0.0526 - val_accuracy: 0.9620 - val_precision: 0.9695
Epoch 3/40
100/100 - 17s - loss: 0.1910 - mae: 0.1014 - accuracy: 0.9305 - precision: 0.9362 - val_loss: 0.1065 - val_mae: 0.0405 - val_accuracy: 0.9660 - val_precision: 0.9679
Epoch 4/40
100/100 - 17s - loss: 0.1947 - mae: 0.0999 - accuracy: 0.9265 - precision: 0.9356 - val_loss: 0.1584 - val_mae: 0.0600 - val_accuracy: 0.9430 - val_precision: 0.9911
Epoch 5/40
100/100 - 17s - loss: 0.1923 - mae: 0.0906 - accuracy: 0.9265 - precision: 0.9312 - val_loss: 0.0821 - val_mae: 0.0423 - val_accuracy: 0.9690 - val_precision: 0.9757
Epoch 6/40
100/100 - 17s - loss: 0.1704 - mae: 0.0838 - accuracy: 0.9365 - precision: 0.9387 - val_loss: 0.0844 - v